# 检查跳跃关联度

## 导入模块

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import feather

## 读入数据

### 日线

In [2]:
price_1d = feather.read_dataframe('../data/StockPriceK1d_20240630.feather')
price_1d = price_1d[(price_1d['date'] >= '2019-01-01') & (price_1d['date'] <= '2024-12-31')]
price_1d = price_1d[(price_1d['ret'] > 0.01) | (price_1d['ret'] < -0.01)]

### 跳跃

In [3]:
jump = feather.read_dataframe('../data/jump/jump.feather')
jump = pd.merge(
    jump,
    price_1d[['issue', 'date']],
    on=['issue', 'date'],
    how='inner'
)

### 调仓日和对应的构建关联度开始时间 (120 天前)

In [4]:
trade_date = price_1d['date'].sort_values().unique()
mes = pd.date_range(start=trade_date[0], end=trade_date[-1]+pd.DateOffset(months=1), freq='1ME')
adj_date = np.array([], dtype=np.datetime64)
calc_start_date = np.array([], dtype=np.datetime64)

for me in mes:
    trade_date_before = trade_date[trade_date <= me]
    ad = trade_date_before[-1]
    adj_date = np.append(adj_date, ad)
    if (len(trade_date_before) > 120):
        csd = trade_date_before[-120]
    else:
        csd = trade_date_before[0]
    calc_start_date = np.append(calc_start_date, csd)

adj_date[-1] = pd.Timestamp('2024-06-28')

In [5]:
ad = adj_date[7]
ad_str = ad.strftime('%Y%m%d')
csd = calc_start_date[7]
print(ad, csd)

2019-08-30 00:00:00 2019-03-11 00:00:00


### 跳跃关联度

In [12]:
corr_num = feather.read_dataframe(f'../data/corr/corr_num_{ad_str}.feather')
corr_size = feather.read_dataframe(f'../data/corr/corr_size_{ad_str}.feather')

## 检查

In [7]:
issue_i = '000001'
issue_j = '000002'
idx_date = (jump['date'] >= csd) & (jump['date'] <= ad)

### 股票 i, j 的跳跃

In [24]:
jump_i = jump[idx_date & (jump['issue'] == issue_i)].copy()
jump_i

,issue,date,jump,ret_jump,year_mon
11,000001,2019-03-29,True,0.019315,201903
12,000001,2019-04-01,True,0.020062,201904
13,000001,2019-04-02,True,-0.011291,201904
14,000001,2019-04-11,True,0.015179,201904
15,000001,2019-04-15,True,0.018773,201904
16,000001,2019-04-30,True,-0.066547,201904
17,000001,2019-05-09,True,0.014694,201905
18,000001,2019-05-10,True,-0.027991,201905
19,000001,2019-05-14,True,0.028101,201905
20,000001,2019-05-15,True,0.010097,201905


In [25]:
jump_j = jump[idx_date & (jump['issue'] == issue_j)].copy()
jump_j

,issue,date,jump,ret_jump,year_mon
360,000002,2019-03-14,True,0.011929,201903
361,000002,2019-03-18,True,0.013702,201903
362,000002,2019-03-20,True,0.015409,201903
363,000002,2019-03-22,True,-0.012063,201903
364,000002,2019-03-25,True,0.004684,201903
365,000002,2019-03-29,True,0.026406,201903
366,000002,2019-04-08,True,0.016353,201904
367,000002,2019-04-11,True,0.034205,201904
368,000002,2019-04-12,True,0.021636,201904
369,000002,2019-04-19,True,-0.010583,201904


### 以股票 i 为焦点的关联跳跃

| jump_date_i | direction | corr_jump_date_j |
| :---------: | :-------: | :--------------: |
| 3.29        | +         | 3.29             |
| 4.1         | +         |                  |
| 4.2         | -         |                  |
| 4.11        | +         | 4.11             |
| 4.15        | +         |                  |
| 4.30        | -         | 4.30             |
| 5.9         | +         | 5.8              |
| 5.10        | -         | 5.10             |
| 5.14        | +         |                  |
| 5.15        | +         |                  |
| 5.17        | -         | 5.17             |
| 5.22        | -         |                  |
| 5.30        | -         |                  |
| 6.11        | +         | 6.11             |
| 6.18        | +         |                  |
| 6.19        | -         |                  |
| 6.21        | -         |                  |
| 6.27        | +         |                  |
| 7.2         | +         | 7.3              |
| 7.3         | -         |                  |
| 7.16        | -         |                  |
| 7.22        | -         | 7.23             |
| 7.31        | -         | 7.31             |
| 8.2         | -         | 8.2              |
| 8.5         | -         | 8.5              |
| 8.7         | +         | 8.7              |
| 8.8         | +         | 8.8              |
| 8.12        | +         | 8.13             |
| 8.13        | +         | 8.13             |
| 8.21        | -         |                  |
| 8.23        | -         |                  |
| 8.26        | -         |                  |

In [14]:
print(corr_num.loc[issue_i, issue_j], corr_size.loc[issue_i, issue_j])

0.5 0.5248122758284149


In [17]:
dates = np.array([
    '2019-03-29', '2019-04-01', '2019-04-02', '2019-04-11', '2019-04-15',
    '2019-04-30', '2019-05-09', '2019-05-10', '2019-05-14', '2019-05-15',
    '2019-05-17', '2019-05-22', '2019-05-30', '2019-06-11', '2019-06-18',
    '2019-06-19', '2019-06-21', '2019-06-27', '2019-07-02', '2019-07-03',
    '2019-07-16', '2019-07-22', '2019-07-31', '2019-08-02', '2019-08-05',
    '2019-08-07', '2019-08-08', '2019-08-12', '2019-08-13', '2019-08-21',
    '2019-08-23', '2019-08-26'
], dtype=np.datetime64)
dates_corr = np.array([
    '2019-03-29', '2019-04-11', '2019-04-30', '2019-05-09', '2019-05-10',
    '2019-05-17', '2019-06-11', '2019-07-02', '2019-07-22', '2019-07-31',
    '2019-08-02', '2019-08-05', '2019-08-07', '2019-08-08', '2019-08-12',
    '2019-08-13'
], dtype=np.datetime64)

In [18]:
len(dates_corr) / len(dates)

0.5

In [29]:
jump_i['abs_ret_jump'] = jump_i['ret_jump'].apply(np.abs)
jump_i.loc[jump_i['date'].isin(dates_corr), 'abs_ret_jump'].sum() / jump_i['abs_ret_jump'].sum()

np.float64(0.524812275828415)